In [12]:
import json
import pandas as pd
import numpy as np
import os
import base64
from openai import AzureOpenAI

import fitz  # PyMuPDF
import io
import base64
from PIL import Image
import os
import json
from tenacity import retry, stop_after_attempt

In [9]:
!pip install tenacity

### Reference Links
- Base package: https://github.com/openai/openai-python
- 

### Utils

In [2]:
def convert_pdf_to_base64_images(pdf_path, output_dir=None):
    """
    Convert each page of a PDF to a base64 encoded PNG image.
    
    Args:
        pdf_path (str): Path to the PDF file
        output_dir (str, optional): Directory to save the images. If None, images are not saved to disk.
        
    Returns:
        list: List of dictionaries with page number and base64 encoded image
    """
    # Create output directory if it doesn't exist and was specified
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open the PDF
    pdf_document = fitz.open(pdf_path)
    result = []
    
    # Process each page
    for page_num, page in enumerate(pdf_document):
        # Render page to an image
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))  # 300 dpi
        
        # Convert pixmap to PIL Image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Save to a bytes buffer
        img_buffer = io.BytesIO()
        img.save(img_buffer, format="PNG")
        img_buffer.seek(0)
        
        # Get base64 encoded string
        b64_image = base64.b64encode(img_buffer.getvalue()).decode("utf-8")
        
        # Save image to disk if output_dir is specified
        if output_dir:
            img_path = os.path.join(output_dir, f"page_{page_num+1}.png")
            with open(img_path, "wb") as img_file:
                img_file.write(img_buffer.getvalue())
        
        result.append({
            "page_number": page_num + 1,
            "base64_image": b64_image
        })
    
    pdf_document.close()
    return result

In [8]:
convert_pdf_to_base64_images(
    pdf_path="F:\github\expense-report-parser\.store\EStatement_M3_574529180_352701501384.pdf",
    output_dir=".test"
)

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\ashis\AppData\Local\Temp\ipykernel_13240\3254736313.py:2: SyntaxWarning: invalid escape sequence '\g'
  pdf_path="F:\github\expense-report-parser\.store\EStatement_M3_574529180_352701501384.pdf",


[{'page_number': 1,
  'base64_image': '...'},
 {'page_number': 2,
  'base64_image': '...'},
 {'page_number': 3,
  'base64_image': '...'}]

In [6]:
import pymupdf

# # Open some document, for example a PDF (could also be EPUB, XPS, etc.)
# doc = pymupdf.open("F:\github\expense-report-parser\.store\EStatement_M3_574529180_352701501384.pdf")

# # Load a desired page. This works via 0-based numbers
# page = doc[0]  # this is the first page

# # Look for tables on this page and display the table count
# tabs = doc[1].find_tables()
# print(f"{len(tabs.tables)} table(s) on {page}")

### LLM - Project Final


In [7]:
endpoint = "https://azflearninghub3689401319.openai.azure.com/"
model_name = "gpt-4o"
deployment = "gpt-4o"

subscription_key = "5Ma7PJCmGONJk3nGcwt6L1V2Hb5c0dMr1YlADOTv7EitGIwBccNcJQQJ99BEACHYHv6XJ3w3AAAAACOGvjhR"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

In [17]:
retry(
    stop=stop_after_attempt(3),
    retry_error_callback=lambda retry_state: None    
)
def get_llm_output(client, page_content, deployment, idx):
    print(f"Trying to extract page {idx}")
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"""
                1. Can you extract the transactions from this data: {page_content}
                2. Always return the transactions in JSON format. The response content itself should be valid JSON string. DONT INCLUDE any other text like ```json which wont allow it to be parsed as json
                """,
            }
        ],
        model=deployment
    )


    data = response.choices[0].message.content
    return json.loads(data)

In [19]:
icici_statement_pdf = "F:\github\expense-report-parser\.store\ICICI_June_14_2025.pdf"
doc = pymupdf.open(icici_statement_pdf)
transactions = []

# Read PDF content (Text) -> LLM -> Extract transactions

for idx, page in enumerate(doc):
    print(f"At page {idx}")
    page_content = page.get_text()
    
    data= get_llm_output(client, page_content, deployment, idx)

    if data is None:
        print(f"Ignoring page : {idx}")
        continue
    transactions = [*transactions, data]



<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\ashis\AppData\Local\Temp\ipykernel_25900\2096510547.py:1: SyntaxWarning: invalid escape sequence '\g'
  icici_statement_pdf = "F:\github\expense-report-parser\.store\ICICI_June_14_2025.pdf"


At page 0
Trying to extract page 0
At page 1
Trying to extract page 1
At page 2
Trying to extract page 2
At page 3
Trying to extract page 3
At page 4
Trying to extract page 4
At page 5
Trying to extract page 5
At page 6
Trying to extract page 6
At page 7
Trying to extract page 7
At page 8
Trying to extract page 8
At page 9
Trying to extract page 9
At page 10
Trying to extract page 10
At page 11
Trying to extract page 11
At page 12
Trying to extract page 12
At page 13
Trying to extract page 13
At page 14
Trying to extract page 14
At page 15
Trying to extract page 15
At page 16
Trying to extract page 16
At page 17
Trying to extract page 17
At page 18
Trying to extract page 18
At page 19
Trying to extract page 19
At page 20
Trying to extract page 20
At page 21
Trying to extract page 21
At page 22
Trying to extract page 22


In [22]:
len(transactions)

23

In [25]:
transactions[1]

[{'date': '05-11-2024',
  'mode': 'ACH/NSEClearingLimited',
  'particulars': 'ICIC7012105244002612/2879465423',
  'deposit': 15000.0,
  'withdrawal': None,
  'balance': 124701.45},
 {'date': '05-11-2024',
  'mode': 'ACH/NSEClearingLimited',
  'particulars': 'ICIC7012105244002612/2879448658',
  'deposit': 10000.0,
  'withdrawal': None,
  'balance': 114701.45},
 {'date': '05-11-2024',
  'mode': 'ACH/NSEClearingLimited',
  'particulars': 'ICIC7012105244002612/2879680492',
  'deposit': 15000.0,
  'withdrawal': None,
  'balance': 99701.45},
 {'date': '05-11-2024',
  'mode': 'ACH/NSEClearingLimited',
  'particulars': 'ICIC7012105244002612/2879622426',
  'deposit': 10000.0,
  'withdrawal': None,
  'balance': 89701.45},
 {'date': '05-11-2024',
  'mode': 'CMS TRANSACTION',
  'particulars': 'CMS/TCS LTD/SECOND INTERIM DIVIDEND 2024-25',
  'deposit': 20.0,
  'withdrawal': None,
  'balance': 89721.45},
 {'date': '05-11-2024',
  'mode': 'UPI',
  'particulars': 'swiggy.rzp@axis/Pay via Razorpa/Axis 

In [ ]:
# transactions = [*transactions, *json.loads(data)]

In [28]:
filtered_transactions = [txn for txn in transactions if type(txn)==list]
ignored_txns = [txn for txn in transactions if type(txn)!=list]
print(len(filtered_transactions), len(ignored_txns))

21 2


In [41]:
final_txns = []
for x in filtered_transactions: final_txns.extend(x)

In [44]:
df = pd.DataFrame.from_records(final_txns)
df.head()

,date,mode,particulars,deposits,withdrawals,balance,deposit,withdrawal,DATE,MODE,...,Deposit Amount (INR ),Balance (INR ),TransactionID,Date,ValueDate,Type,Description,Amount,Charges,Balance
0,01-11-2024,B/F,,0,0,177588.69,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-11-2024,UPI,hegdeganesh051@/UPI/Union Bank of I/4672332341...,0,21000.0,156588.69,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-11-2024,UPI,vijajanandtrave/UPI/ICICI Bank LTD /4671788775...,707.56,0,157296.25,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-11-2024,UPI,amazonpaygiftca/Request from Am/Axis Bank Ltd/...,37.8,0,157258.45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,02-11-2024,UPI,8086190005@okbi/UPI/Axis Bank Ltd/430708059113...,0,300.0,156958.45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df.sort_values(by="date").to_csv("final_data_July_14_2025.csv",index=None)

In [ ]:
# df.to_csv("data.csv", index=None)

### Rough Space

In [195]:
df["particulars"].tolist()

['credpay.dineout/payment on CRED/AXIS BANK/546618839785/ACD3117222621ynXwmrg/',
 'credpay.swiggy@/payment on CRED/AXIS BANK/546644323215/ACD3117924775DvzaakK/',
 'zeptonow.bdpg@i/Paid via CRED/ICICI Bank LTD/510088562026/YCD01JRFXP0CZ3WEEKW5VD2SV4FCVVlBZ',
 'rdeva1365@okhdf/Payment from Ph/HDFC BANK LTD/115859252254/IBLcedd78eeada14e069481d8ee4a6956b5/',
 'paytm-76881028@/Payment from Ph/YES BANK LIMITE/238791412548/IBLf6b07a74c7e64632a44ba978698f790d',
 'pinelabs.103884/Payment for v1/HDFC BANK LTD/546741079814/YCD01JRJG401D2ZMSTDV6CN3S1FQJ0ZvbK0/',
 'credpay.swiggy@/payment on CRED/AXIS BANK/546756948277/ACD3120619897mRlbXzx/',
 '70222027@ybl/UPI/BANK OF BARODA/546878405855/ICI80badff0788a48fa929f911245a92941/',
 'vilitiktsuquvil/Payment from Ph/AXIS BANK/659898798036/AXL07dbc20edf7e4951bdda1ed34f2b99ba/',
 'paytmqr4oxq8x5i/Payment from Ph/YES BANK LIMITE/258849561408/IBL9668f93d2ee04ed587ac02c80974a469',
 '7635889498@ybl/UPI/State Bank Of I/546862725834/ICI1a5ec3fe4f7146d0bd1496a5c

In [197]:
response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"""
                1. Here are the list of transaction details/descriptions derived from a bank statemment: {df["particulars"].tolist()}
                2. I want you to categorize these into fever number of broad level categories that you can understand from the descriptions
                3. Return a json as list of mappings like {{"descriptions":'mapped_category'}}
                4. Always return the mapping in JSON format. The response content itself should be valid JSON string. DONT INCLUDE any other text like ```json which wont allow it to be parsed as json
                """,
            }
        ],
        model=deployment
    )

data = response.choices[0].message.content

In [199]:
response_data = json.loads(data)
response_data

[{'credpay.dineout/payment on CRED/AXIS BANK/546618839785/ACD3117222621ynXwmrg/': 'Food & Dining'},
 {'credpay.swiggy@/payment on CRED/AXIS BANK/546644323215/ACD3117924775DvzaakK/': 'Food & Dining'},
 {'zeptonow.bdpg@i/Paid via CRED/ICICI Bank LTD/510088562026/YCD01JRFXP0CZ3WEEKW5VD2SV4FCVVlBZ': 'Groceries'},
 {'rdeva1365@okhdf/Payment from Ph/HDFC BANK LTD/115859252254/IBLcedd78eeada14e069481d8ee4a6956b5/': 'Peer-to-Peer Payment'},
 {'paytm-76881028@/Payment from Ph/YES BANK LIMITE/238791412548/IBLf6b07a74c7e64632a44ba978698f790d': 'Peer-to-Peer Payment'},
 {'pinelabs.103884/Payment for v1/HDFC BANK LTD/546741079814/YCD01JRJG401D2ZMSTDV6CN3S1FQJ0ZvbK0/': 'Retail'},
 {'credpay.swiggy@/payment on CRED/AXIS BANK/546756948277/ACD3120619897mRlbXzx/': 'Food & Dining'},
 {'70222027@ybl/UPI/BANK OF BARODA/546878405855/ICI80badff0788a48fa929f911245a92941/': 'Peer-to-Peer Payment'},
 {'vilitiktsuquvil/Payment from Ph/AXIS BANK/659898798036/AXL07dbc20edf7e4951bdda1ed34f2b99ba/': 'Peer-to-Peer Pa

In [ ]:
# response_data = [{'credpay.dineout/payment on CRED/AXIS BANK/546618839785/ACD3117222621ynXwmrg/': 'Food & Dining'},
#  {'credpay.swiggy@/payment on CRED/AXIS BANK/546644323215/ACD3117924775DvzaakK/': 'Food & Dining'},
#  {'zeptonow.bdpg@i/Paid via CRED/ICICI Bank LTD/510088562026/YCD01JRFXP0CZ3WEEKW5VD2SV4FCVVlBZ': 'Groceries'},
#  {'rdeva1365@okhdf/Payment from Ph/HDFC BANK LTD/115859252254/IBLcedd78eeada14e069481d8ee4a6956b5/': 'Peer-to-Peer Payment'},
#  {'paytm-76881028@/Payment from Ph/YES BANK LIMITE/238791412548/IBLf6b07a74c7e64632a44ba978698f790d': 'Peer-to-Peer Payment'},
#  {'pinelabs.103884/Payment for v1/HDFC BANK LTD/546741079814/YCD01JRJG401D2ZMSTDV6CN3S1FQJ0ZvbK0/': 'Retail'},
#  {'credpay.swiggy@/payment on CRED/AXIS BANK/546756948277/ACD3120619897mRlbXzx/': 'Food & Dining'},
#  {'70222027@ybl/UPI/BANK OF BARODA/546878405855/ICI80badff0788a48fa929f911245a92941/': 'Peer-to-Peer Payment'},
#  {'vilitiktsuquvil/Payment from Ph/AXIS BANK/659898798036/AXL07dbc20edf7e4951bdda1ed34f2b99ba/': 'Peer-to-Peer Payment'},
#  {'paytmqr4oxq8x5i/Payment from Ph/YES BANK LIMITE/258849561408/IBL9668f93d2ee04ed587ac02c80974a469': 'Retail'},
#  {'7635889498@ybl/UPI/State Bank Of I/546862725834/ICI1a5ec3fe4f7146d0bd1496a5c9ef7d69/': 'Peer-to-Peer Payment'},
#  {'actcorp.payu@in/Upi Transaction/INDUSIND BANKL/100106200649/INDB32A373E391E7F244E063F87C180ADFF': 'Utilities'},
#  {'BIL/000997366236/ICICI BANK CREDIT CA/431581269992': 'Credit Card Payment'},
#  {'uberindiasystem/PaymenttoUBERIN/ICICI Bank LTD/303797540192/IBLec1cd417d2c14bf1867d4d12669e6d16': 'Transportation'},
#  {'8971052659@ybl/Payment from Ph/UNION BANK OF I/304466899809/IBL5c80eaee16dd435d8a54efdb0b64bde5/': 'Peer-to-Peer Payment'},
#  {'zeptomarketplac/home/AXIS BANK/510666889056/YCD01JRZ135XC0A0272H5TCS7VJKGPGwZYL/': 'Groceries'},
#  {'laskarikbalhuss/Payment from Ph/State Bank Of I/260059476482/IBL814766d8e5ad479d83d485797459a267': 'Peer-to-Peer Payment'},
#  {'NEFT-CITIN52025041750620814-MICROSOFT CORPN I PVTLTD-DIFFERENTIAL PAYMENT-0001940015-CITI0000002': 'Salary or Professional Services'},
#  {'paytmqr14z3m9@p/Payment from Ph/YES BANK LIMITE/328440198522/IBL18760f26755d454c85186ae803738806': 'Retail'},
#  {'paytmqr28100505/Payment from Ph/YES BANK LIMITE/807443552783/IBL81470df8ef674263bc567dacb88b63b5': 'Retail'},
#  {'BHARATPE.900720/Pay to BharatPe/FEDERAL BANK/902431848539/IBL99f64e8a5a4a4638bc9ab49919a843d6/': 'Retail'},
#  {'paytm-76881028@/Payment from Ph/YES BANK LIMITE/083603262759/IBL1e044834968e4a54ad1f03bccde4863c': 'Peer-to-Peer Payment'},
#  {'credpay.swiggy@/payment on CRED/AXIS BANK/547664884823/ACD3138490096LYPbvzA/': 'Food & Dining'},
#  {'sunitapednekar3/Ashish Salaskar/State Bank OfI/511039177292/ICI6fe9130ab2b142cc9bd7271b4ea9e679': 'Peer-to-Peer Payment'},
#  {'7638077442@ybl/Payment from Ph/State Bank Of I/477002714689/IBL8d64e6b477d24838889ec0aef1ac7cc5/': 'Peer-to-Peer Payment'},
#  {'TO FD no. 352713001794': 'Savings & Investments'},
#  {'actpl.billdesk@/Pay/HDFC BANK LTD/287730584449/IBL16aec94202324338a8fd9e838e1b995a/': 'Utilities'},
#  {'7026612134@ybl/Payment from Ph/HDFC BANK LTD/820986803071/IBL58b3fd450e784e6380dd7900889b0ff9/': 'Peer-to-Peer Payment'},
#  {'playstore@axisb/MandateExecute/AXIS BANK/488259901155/AXI7e07cf71e6de404787bd1426d85c84a0/': 'Subscription/Services'},
#  {'zerodhabroking@/586980847463746/HDFC BANK LTD/511638450967/ICI77b2986812ce4fa18d2960c674d47a16': 'Investments'},
#  {'zeptomarketplac/ZEPTOMARKETPLAC/AIRTEL PAYMENTS/548217314718/YCD01JSS6TCVWB64GJ0MS6X1V8QBBVygxje': 'Groceries'},
#  {'paytm-jiomobili/Payment from Ph/YES BANK LIMITE/772161610572/PYTM50426806086313395225XXXXXX': 'Utilities'},
#  {'CITIN52025042854233163-MICROSOFT CORPN I PVTLTD-SALARY PAYMENT FOR A-0001940015-CITI0000002': 'Salary or Professional Services'},
#  {'MMT/IMPS/511821107997/ASHOK SALA/UBIN0901253': 'Peer-to-Peer Payment'},
#  {'9989577110@axl/Payment from Ph/State Bank Of I/872541332908/IBL48621513ac4f4735be856ec949fe9aa8': 'Peer-to-Peer Payment'},
#  {'QR916363009839-/Payment from Ph/UNION BANK OFI/237641515406/IBL4924aa7c63fe47be89b2e6b4d33ab6d3': 'Peer-to-Peer Payment'},
#  {'9535191887@axl/Payment from Ph/State Bank Of I/934389475919/IBL95f84ffbdc0347ce8fa9dd0f67cac9fb': 'Peer-to-Peer Payment'},
#  {'ashishsalaskar1/UPI/IDFC FIRST BANK/548645565710/ICIdd37ce17410b4444b31ea8e6888ac91d': 'Peer-to-Peer Payment'},
#  {'credpay.swiggy@/payment on CRED/AXIS BANK/545700918477/ACD3085399336LBrevaE/': 'Food & Dining'},
#  {'blinkit104020.r/Blinkit/AIRTEL PAYMENTS/545808243524/AIR1QE64g23XeweFzeT6814729350QPAY02/': 'Groceries'},
#  {'credpay.swiggy1/payment on CRED/AXIS BANK/509243862501/ACD3089580922LbEjeMK/': 'Food & Dining'},
#  {'credpay.swiggy@/payment on CRED/AXIS BANK/509228092632/ACD3090663405jzvppvj/': 'Food & Dining'},
#  {'blinkit104020.r/Blinkit/AIRTEL PAYMENTS/509250034188/AIR1QEDbvpoOq5nypgT0672438519QPAY02/': 'Groceries'},
#  {'vijaykumarreddy/UPI/ICICI Bank/509274949620/ICI75a4908e9ed84da694db6bbe5b0f0e8e/': 'Peer-to-Peer Payment'},
#  {'ppuja1085@okhdf/Payment from Ph/State Bank OfI/585283113213/IBL9ef06a2b35b4461b9a181cd78f69f471': 'Peer-to-Peer Payment'},
#  {'CITIN52025040445754484-WM GLOBAL TECHNOLOGY SERVICES I P L-WMGTS FFS PAY MAR 20-0520551018-CIT': 'Salary or Professional Services'},
#  {'NSEClearingLimited/ICIC7012105244002612/3142726143': 'Investments'},
#  {'NSEClearingLimited/ICIC7012105244002612/3142531691': 'Investments'},
#  {'NSEClearingLimited/ICIC7012105244002612/3142805383': 'Investments'},
#  {'NSEClearingLimited/ICIC7012105244002612/3142513134': 'Investments'},
#  {'NSEClearingLimited/ICIC7012105244002612/3142345145': 'Investments'},
#  {'credpay.swiggy@/payment on CRED/AXIS BANK/509592586575/ACD3101903163ymvYDBr/': 'Food & Dining'},
#  {'credpay.swiggy1/payment on CRED/AXIS BANK/546131249332/ACD3102492735LzagMnk/': 'Food & Dining'},
#  {'credpay.swiggy@/payment on CRED/AXIS BANK/509705176535/ACD3109642217NWlWRbM/': 'Food & Dining'}]

In [10]:
desc_mappings = [{"desc": list(x.keys())[0], "category": list(x.values())[0]} for x in response_data]

In [15]:
df_mappings = pd.DataFrame.from_records(desc_mappings)
df_mappings.head()

,desc,category
0,credpay.dineout/payment on CRED/AXIS BANK/5466...,Food & Dining
1,credpay.swiggy@/payment on CRED/AXIS BANK/5466...,Food & Dining
2,zeptonow.bdpg@i/Paid via CRED/ICICI Bank LTD/5...,Groceries
3,rdeva1365@okhdf/Payment from Ph/HDFC BANK LTD/...,Peer-to-Peer Payment
4,paytm-76881028@/Payment from Ph/YES BANK LIMIT...,Peer-to-Peer Payment


In [45]:
df = pd.read_csv("final_data_July_14_2025.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'final_data_July_14_2025.csv'

In [18]:
pd.merge(left=df, right=df_mappings, how="left", left_on="particulars", right_on="desc").to_csv("data_categorized.csv",index=None)

In [20]:
df_final = pd.read_csv("data_categorized.csv")
df_final.head(10)

,date,mode,particulars,deposits,withdrawals,balance,desc,category
0,01-04-2025,UPI,credpay.swiggy@/payment on CRED/AXIS BANK/5457...,NaN,187.00,286683.07,credpay.swiggy@/payment on CRED/AXIS BANK/5457...,Food & Dining
1,02-04-2025,UPI,vijaykumarreddy/UPI/ICICI Bank/509274949620/IC...,NaN,29000.00,256494.07,vijaykumarreddy/UPI/ICICI Bank/509274949620/IC...,Peer-to-Peer Payment
2,02-04-2025,UPI,blinkit104020.r/Blinkit/AIRTEL PAYMENTS/509250...,NaN,237.00,285494.07,blinkit104020.r/Blinkit/AIRTEL PAYMENTS/509250...,Groceries
3,02-04-2025,UPI,credpay.swiggy@/payment on CRED/AXIS BANK/5092...,NaN,182.00,285731.07,credpay.swiggy@/payment on CRED/AXIS BANK/5092...,Food & Dining
4,02-04-2025,UPI,credpay.swiggy1/payment on CRED/AXIS BANK/5092...,NaN,288.00,285913.07,credpay.swiggy1/payment on CRED/AXIS BANK/5092...,Food & Dining
5,02-04-2025,UPI,blinkit104020.r/Blinkit/AIRTEL PAYMENTS/545808...,NaN,482.00,286201.07,blinkit104020.r/Blinkit/AIRTEL PAYMENTS/545808...,Groceries
6,04-04-2025,NEFT,CITIN52025040445754484-WM GLOBAL TECHNOLOGY SE...,452817.00,NaN,709259.07,CITIN52025040445754484-WM GLOBAL TECHNOLOGY SE...,Salary or Professional Services
7,04-04-2025,UPI,ppuja1085@okhdf/Payment from Ph/State Bank OfI...,NaN,52.00,256442.07,ppuja1085@okhdf/Payment from Ph/State Bank OfI...,Peer-to-Peer Payment
8,05-04-2025,UPI,credpay.swiggy@/payment on CRED/AXIS BANK/5095...,NaN,180.00,649079.07,credpay.swiggy@/payment on CRED/AXIS BANK/5095...,Food & Dining
9,05-04-2025,ACH,NSEClearingLimited/ICIC7012105244002612/314234...,NaN,10000.00,649259.07,NSEClearingLimited/ICIC7012105244002612/314234...,Investments


## Analysis

In [47]:
df = pd.read_csv('final_data_July_14_2025.csv')
df.head()

,date,mode,particulars,deposits,withdrawals,balance,deposit,withdrawal,DATE,MODE,...,Deposit Amount (INR ),Balance (INR ),TransactionID,Date,ValueDate,Type,Description,Amount,Charges,Balance
0,01-01-2025,B/F,NaN,NaN,NaN,188401.18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-02-2025,UPI,upiswiggy@icici/Payment for Swi/ICICI Bank LTD...,NaN,148.00,213624.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-02-2025,UPI,swiggyupi@axb/remarks/AXIS BANK/539898457109/A...,NaN,164.00,213772.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-02-2025,B/F,NaN,NaN,NaN,213936.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-03-2025,UPI,blinkit104020.r/Blinkit/AIRTEL PAYMENTS/542612...,NaN,1107.0,633739.91,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
